# J/ψ → K⁰Σ⁺p̅

```{autolink-concat}
```

In [ ]:
# pyright: reportPrivateUsage=false
from __future__ import annotations

import itertools
from typing import Iterable

import qrules
import sympy as sp
from ampform.dynamics import EnergyDependentWidth, formulate_form_factor
from IPython.display import Latex, Markdown

from ampform_dpd import (
    DalitzPlotDecompositionBuilder,
    _get_particle,
    simplify_latex_rendering,
)
from ampform_dpd.decay import (
    IsobarNode,
    Particle,
    ThreeBodyDecay,
    ThreeBodyDecayChain,
)
from ampform_dpd.io import as_markdown_table, aslatex
from ampform_dpd.spin import filter_parity_violating_ls, generate_ls_couplings

simplify_latex_rendering()

## Decay definition

We follow [this example](https://qrules.readthedocs.io/en/0.9.7/usage.html#investigate-intermediate-resonances), which was generated with QRules, and leave out the $K$-resonances:

![](https://qrules.readthedocs.io/en/0.9.7/_images/usage_9_0.svg)

In [ ]:
PDG = qrules.load_pdg()
PARTICLE_DB = {
    p.name: Particle(
        name=p.name,
        latex=p.latex,
        spin=p.spin,
        parity=int(p.parity),
        mass=p.mass,
        width=p.width,
    )
    for p in PDG
    if p.parity is not None
}
Jpsi = PARTICLE_DB["J/psi(1S)"]
K = PARTICLE_DB["K0"]
Σ = PARTICLE_DB["Sigma+"]
pbar = PARTICLE_DB["p~"]
PARTICLE_TO_ID = {Jpsi: 0, K: 1, Σ: 2, pbar: 3}
Markdown(as_markdown_table(list(PARTICLE_TO_ID)))

In [ ]:
resonance_names = [
    "Sigma(1385)~-",
    "Sigma(1660)~-",
    "Sigma(1670)~-",
    "Sigma(1750)~-",
    "Sigma(1775)~-",
    "Sigma(1910)~-",
    "N(1440)+",
    "N(1520)+",
    "N(1535)+",
    "N(1650)+",
    "N(1675)+",
    "N(1700)+",
    "N(1710)+",
    "N(1720)+",
]
resonances = [PARTICLE_DB[name] for name in resonance_names]
Markdown(as_markdown_table(resonances))

In [ ]:
def load_three_body_decay(
    resonance_names: Iterable[str],
    particle_definitions: dict[str, Particle],
    min_ls: bool = True,
) -> ThreeBodyDecay:
    resonances = [particle_definitions[name] for name in resonance_names]
    chains: list[ThreeBodyDecayChain] = []
    for res in resonances:
        chains.extend(_create_isobar(res, min_ls))
    return ThreeBodyDecay(
        states={state_id: particle for particle, state_id in PARTICLE_TO_ID.items()},
        chains=tuple(chains),
    )


def _create_isobar(resonance: Particle, min_ls: bool) -> list[ThreeBodyDecayChain]:
    if resonance.name.startswith("Sigma"):
        child1, child2, spectator = pbar, K, Σ
    elif resonance.name.startswith("N"):
        child1, child2, spectator = K, Σ, pbar
    elif resonance.name.startswith("K"):
        child1, child2, spectator = Σ, pbar, K
    else:
        raise NotImplementedError
    prod_ls_couplings = _generate_ls(
        Jpsi, resonance, spectator, conserve_parity=False
    )
    dec_ls_couplings = _generate_ls(resonance, child1, child2, conserve_parity=True)
    if min_ls:
        decay = IsobarNode(
            parent=Jpsi,
            child1=IsobarNode(
                parent=resonance,
                child1=child1,
                child2=child2,
                interaction=min(dec_ls_couplings),
            ),
            child2=spectator,
            interaction=min(prod_ls_couplings),
        )
        return [ThreeBodyDecayChain(decay)]
    chains = []
    for dec_ls, prod_ls in itertools.product(dec_ls_couplings, prod_ls_couplings):
        decay = IsobarNode(
            parent=Jpsi,
            child1=IsobarNode(
                parent=resonance,
                child1=child1,
                child2=child2,
                interaction=dec_ls,
            ),
            child2=spectator,
            interaction=prod_ls,
        )
        chains.append(ThreeBodyDecayChain(decay))
    return chains


def _generate_ls(
    parent: Particle, child1: Particle, child2: Particle, conserve_parity: bool
) -> list[tuple[int, sp.Rational]]:
    ls = generate_ls_couplings(parent.spin, child1.spin, child2.spin)
    if conserve_parity:
        return filter_parity_violating_ls(
            ls, parent.parity, child1.parity, child2.parity
        )
    return ls


DECAY = load_three_body_decay(
    resonance_names,
    particle_definitions=PARTICLE_DB,
    min_ls=True,
)
Latex(aslatex(DECAY, with_jp=True))

## Lineshapes for dynamics

:::{note}
As opposed to [AmpForm](https://ampform.rtfd.io), AmpForm-DPD defines dynamics over the **entire decay chain**, not a single isobar node. The dynamics classes and the corresponding builders would have to be extended to implement other dynamics lineshapes.
:::

In [ ]:
def formulate_breit_wigner_with_ff(
    decay_chain: ThreeBodyDecayChain,
) -> tuple[sp.Expr, dict[sp.Symbol, float]]:
    production_node = decay_chain.decay
    assert isinstance(
        production_node.child1, IsobarNode
    ), "Not a 3-body isobar decay"
    decay_node = production_node.child1

    s = _get_mandelstam_s(decay_chain)
    parameter_defaults = {}
    production_ff, new_pars = _create_form_factor(s, production_node)
    parameter_defaults.update(new_pars)
    decay_ff, new_pars = _create_form_factor(s, decay_node)
    parameter_defaults.update(new_pars)
    breit_wigner, new_pars = _create_breit_wigner(s, decay_node)
    return (
        production_ff * decay_ff * breit_wigner,
        parameter_defaults,
    )


def _create_form_factor(
    s: sp.Symbol, isobar: IsobarNode
) -> tuple[sp.Expr, dict[sp.Symbol, float]]:
    assert isobar.interaction is not None, "Need LS-couplings"
    inv_mass = _create_mass_symbol(isobar.parent)
    outgoing_state_mass1 = _create_mass_symbol(isobar.child1)
    outgoing_state_mass2 = _create_mass_symbol(isobar.child2)
    meson_radius = _create_meson_radius_symbol(isobar.parent)
    form_factor = formulate_form_factor(
        s=inv_mass**2,
        m_a=outgoing_state_mass1,
        m_b=outgoing_state_mass2,
        angular_momentum=isobar.interaction.L,
        meson_radius=meson_radius,
    )
    parameter_defaults = {
        meson_radius: 1,
    }
    return form_factor, parameter_defaults


def _create_breit_wigner(
    s: sp.Symbol, isobar: IsobarNode
) -> tuple[sp.Expr, dict[sp.Symbol, float]]:
    assert isobar.interaction is not None, "Need LS-couplings"
    outgoing_state_mass1 = _create_mass_symbol(isobar.child1)
    outgoing_state_mass2 = _create_mass_symbol(isobar.child2)
    angular_momentum = isobar.interaction.L
    res_mass = _create_mass_symbol(isobar.parent)
    res_width = sp.Symbol(Rf"\Gamma_{{{isobar.parent.latex}}}", nonnegative=True)
    meson_radius = _create_meson_radius_symbol(isobar.parent)

    mass_dependent_width = EnergyDependentWidth(
        s=s,
        mass0=res_mass,
        gamma0=res_width,
        m_a=outgoing_state_mass1,
        m_b=outgoing_state_mass2,
        angular_momentum=angular_momentum,
        meson_radius=meson_radius,
    )
    breit_wigner_expr = (res_mass * res_width) / (
        res_mass**2 - s - mass_dependent_width * res_mass * sp.I
    )
    parameter_defaults = {
        res_mass: isobar.parent.mass,
        res_width: isobar.parent.width,
        meson_radius: 1,
    }
    return breit_wigner_expr, parameter_defaults


def _create_meson_radius_symbol(isobar: IsobarNode) -> sp.Symbol:
    if _get_particle(isobar) is Jpsi:
        return sp.Symbol(R"R_{J/\psi}")
    return sp.Symbol(R"R_\mathrm{res}")


def _create_mass_symbol(particle: IsobarNode | Particle) -> sp.Symbol:
    particle = _get_particle(particle)
    state_id = PARTICLE_TO_ID.get(particle)
    if state_id is not None:
        return sp.Symbol(f"m{state_id}", nonnegative=True)
    return sp.Symbol(f"m_{{{particle.latex}}}", nonnegative=True)


def _get_mandelstam_s(decay: ThreeBodyDecayChain) -> sp.Symbol:
    s1, s2, s3 = sp.symbols("sigma1:4", nonnegative=True)
    m1, m2, m3 = map(_create_mass_symbol, [K, Σ, pbar])
    decay_masses = {_create_mass_symbol(p) for p in decay.decay_products}
    if decay_masses == {m2, m3}:
        return s1
    if decay_masses == {m1, m3}:
        return s2
    if decay_masses == {m1, m2}:
        return s3
    raise NotImplementedError(
        f"Cannot find Mandelstam variable for {''.join(decay_masses)}"
    )

## Model formulation

In [ ]:
model_builder = DalitzPlotDecompositionBuilder(DECAY, min_ls=True)
for chain in model_builder.decay.chains:
    model_builder.dynamics_choices.register_builder(
        chain, formulate_breit_wigner_with_ff
    )
model = model_builder.formulate(reference_subsystem=1)
model.intensity

In [ ]:
Latex(aslatex(model.amplitudes))

:::{note}
As opposed to {doc}`/lc2pkpi`, some amplitudes are zero. This is because we do not considered all three subsystems in this example.
:::